<a href="https://colab.research.google.com/github/kanwarpal11/PyhthonTutorial/blob/DevBranch/CompareAudioFiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
import torch
import torchaudio
import numpy as np
from scipy.spatial.distance import cosine

# 🔧 Settings
STORED_AUDIO_DIR = "/content/audiofiles"
UPLOAD_FILE = "Mysamplefile.wav"

# 🎵 Load audio and get mel-spectrogram
def extract_features(filepath):
    waveform, sample_rate = torchaudio.load(filepath)
    transform = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate, n_mels=64)
    mel_spec = transform(waveform)
    features = torch.mean(mel_spec, dim=-1).squeeze().numpy()  # shape: [n_mels]
    # Ensure the features are 1D
    if features.ndim > 1:
        features = features.flatten()
    return features

# 💾 Load stored audio features
def load_audio_database():
    database = []
    for fname in os.listdir(STORED_AUDIO_DIR):
        if fname.endswith(".wav"):
            path = os.path.join(STORED_AUDIO_DIR, fname)
            features = extract_features(path)
            database.append((fname, features))
    return database

# 🔍 Compare and rank similarities
def compare_uploaded_to_database(upload_path):
    upload_features = extract_features(upload_path)
    database = load_audio_database()

    similarities = []
    for name, features in database:
        score = 1 - cosine(upload_features, features)  # cosine similarity
        similarities.append((name, score))

    # Sort by descending similarity
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities

# 🚀 Run
if __name__ == "__main__":
    results = compare_uploaded_to_database(UPLOAD_FILE)
    print("🔍 Similar audio files:")
    for name, score in results[:5]:  # top 5
        print(f"{name}: similarity score = {score:.3f}")

🔍 Similar audio files:
NewSample.wav: similarity score = 0.869


Please upload the audio file "Siya.wav".

In [10]:
from google.colab import files

uploaded = files.upload()

# Assuming the uploaded file is 'Siya.wav'
if 'Siya.wav' in uploaded:
  print("File 'Siya.wav' uploaded successfully.")
else:
  print("Please upload 'Siya.wav' to proceed.")

Saving Siya.wav to Siya (1).wav
Please upload 'Siya.wav' to proceed.
